In [ ]:
import numpy as np
from university_sim_helpers import *

import operator as op
import math
import numpy as np
from collections import Counter

#the following three packages control the plot of the simulated university network
import networkx as nx
from networkx.algorithms import node_classification
from PIL import Image
import matplotlib.pyplot as plt

#the following four needed for run heuristics,
from simulation_helpers import *
from heuristics import *
from optimal_policy import V
import pandas as pd

import os
import time

In [ ]:
p_list = [0.01,0.3,0.5]
q_list = [0,0.3,0.7],

n_list = [100] #fix the number of students in school
#example: for computational sake cannot simulate too large a network
   
L_list = [0,0.25,0.5] #percentage of population to be tested
H = 9
heuristics = [selectRandom, highRisk, highConnection, highBelief, sampleBelief, highBeliefRisk]

num_sim = 50

C=20 #number of classes opened by the school
mu=15.0 #avg number of students per class
sd_list=[3.0,5.0,8.0] #sd of students per class, vary sd to adjust for probability of getting larger class size
M=5#Note that major number should preferably not exceed 25, 
   #due to color vector setup in plot function is only limited to length 25
plug=0.98*1/M
perc_list=[0,0.3,0.7,1]
  
results = pd.DataFrame()

A_storage=np.zeros((sum(n_list),sum(n_list),np.shape(sd_list)[0]*np.shape(perc_list)[0]))
MS_storage=np.zeros((np.shape(sd_list)[0]*np.shape(perc_list)[0],M))
onestart=time.time()
for n in n_list: #fixed, no. of students in school
    j=0
    for perc in perc_list:
        for sd in sd_list:
            plt.clf() #clear the figure
            start = time.time()
            print('Current running perc: ', perc,'; Current running sd: ', sd)
           
            A,MS,CS = AdjacencyUniWMajor(n,C,mu,sd,M,plug,perc)
            A_storage[:,:,j]=A
            MS_storage[j,:]=MS
            j=j+1
           
            plotUniversityNetwork(A,M,MS)
            plt.savefig('UniversityNetwork_perc_'+str(perc)+'_sd'+str(sd)+'.png')
            #given the same adjacency matrix, see how the test heruistics perform
      

            for i in range(num_sim):
                #print('Current perc: ', perc, ' ;Current i; ', i)
                for p in p_list:
                    for q in q_list:
                        for L in L_list:
                            for algo in heuristics:
                                sample_res = sample(algo, 
                                                  np.ones(n).astype(np.bool), 
                                                  H, p, q, int(L*n), A, n)
                                result = pd.DataFrame({'population_n':n,
                                                        'infected': sample_res,
                                                       'h': np.arange(H+1),
                                                       'algo': algo.__name__,
                                                        'p': p,
                                                      'q': q,
                                                      'L': int(L*n),
                                                      'priority_percentage': perc, #input perc to show differences of connection across majors
                                                      'sd':sd,
                                                       'sim_id': i})
                                #print(result)
                                results = results.append(result)

 
            results.to_csv('heuristic_tests_university.csv', index = False)    
            print("total time taken this loop: ", time.time() - start)
   
 print("total time for this simulation: ", time.time() - onestart)

In [ ]:
total_in_storage=np.zeros(np.shape(A_storage)[2])
total_out_storage=np.zeros(np.shape(A_storage)[2])
    
for i in range(np.shape(A_storage)[2]):

    A_test=A_storage[:,:,i]
    MS_test=MS_storage[i,:]
    cumMS_test=np.cumsum(MS_test)
    MS_test_adj=np.insert(cumMS_test,0,0)

    inMajor_con=float(0)
    for j in range(np.shape(MS_test)[0]):
        sub_A=A_test[int(MS_test_adj[j]):(int(MS_test_adj[j+1])-1),int(MS_test_adj[j]):(int(MS_test_adj[j+1])-1)]
        inMajor_con=inMajor_con+float(sum(sum(sub_A)))

    total_in_storage[i]=inMajor_con/2
    total_out_storage[i]=sum(sum(A_test))/2-total_in_storage[i]